In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
#input_file = "output_20200219_2125PM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #double
df = pd.read_csv(input_file, header = 0)
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']
df_failure = df[ (df.FM==1)]

In [39]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','FILESIZE', 'THROUGHPUT','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])
df_dummy['THROUGHPUT']=df_dummy['THROUGHPUT'].str[0:-4]
df_dummy['THROUGHPUT']=pd.to_numeric(df_dummy['THROUGHPUT'], errors='coerce')
df_dummy = df_dummy.replace(np.nan, 0, regex=True)
y_complete=df_dummy['LABEL']
X_complete=df_dummy.drop(['LABEL'],axis=1)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier

In [41]:
df_dummy_cate = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FM','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S'])
y_cate=df_dummy_cate['LABEL']
X_cate=df_dummy_cate.drop(['LABEL'],axis=1)
X_cate[0:1]

,FM,RETRIES,SRC_CacheCal,SRC_CacheFNAL,SRC_CacheNY,SRC_CacheSDSC,SRC_CacheUC,SRC_CacheUNL,SRC_OriginCal,SRC_OriginFNAL,...,SRC_R_OriginUNLRouter,DEST_S_CacheCENICRouter,DEST_S_CacheNYRouter,DEST_S_CacheStarLightRouter,DEST_S_CacheUNLRouter,DEST_S_OriginCalRouter,DEST_S_OriginNCSARouter,DEST_S_OriginSDSCRouter,DEST_S_OriginStarLightRouter,DEST_S_OriginUNLRouter
8,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [43]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
    return correct_class, correct, correct/num 

In [44]:
def print_s_c(c):
    s_c=sorted(c.items(), key=lambda x: x[1], reverse=True)
    num_link=0;num_host=0;link_pos=0;host_pos=0
    for i in s_c:
        name=i[0]
        flag=i[1]
        if 'Link' in name:
            num_link+=1
            if flag==1:
                link_pos+=1
        else:
            num_host+=1
            if flag==1:
                host_pos+=1
    return link_pos*1.0/num_link,host_pos*1.0/num_host

In [45]:
training_dataset={"complete":{"X":X_complete,
                 "y":y_complete},
                 "cate":{"X":X_cate,
                 "y":y_cate}}

In [46]:
def train_dt(classifier):
    df=[df_dummy,df_dummy_cate]
    i=0
    for k, d in training_dataset.items():
        X=d["X"]
        y=d['y']
        clf = classifier
        clf_model=clf.fit(X,y)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_,j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
        i=i+1

In [47]:
clf_mn = MultinomialNB()
clf_gs = GaussianNB()
train_dt(clf_gs)

complete:balanced_accu=0.17833129645208406:F1-Score=0.005426854289100885
Top-1 Accu=0.16666666666666666
Top-2 Accu=0.18181818181818182
Top-3 Accu=0.18181818181818182
cate:balanced_accu=0.21599220466150051:F1-Score=0.006118511877678833
Top-1 Accu=0.24242424242424243
Top-2 Accu=0.2727272727272727
Top-3 Accu=0.2727272727272727


In [12]:
c,r=accuracy(clf_mn, df_dummy, clf_mn.classes_,3)
c,r

0 OriginSDSC_Link17 CENIC_Link0
1 OriginSDSCRouter_Link5 CENIC_Link0
2 CENIC_Link5 CENIC_Link0
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link5']
False
0 OriginSDSC_Link17 CENIC_Link1
1 OriginSDSCRouter_Link5 CENIC_Link1
2 CENIC_Link1 CENIC_Link1
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link1']
True
0 CENIC_Link5 CENIC_Link5
['CENIC_Link5']
True
0 OriginSDSC_Link17 CENIC_Link9
1 OriginSDSCRouter_Link5 CENIC_Link9
2 CENIC_Link9 CENIC_Link9
['OriginSDSC_Link17', 'OriginSDSCRouter_Link5', 'CENIC_Link9']
True
0 ESNET_Link4 CacheCENICRouter_Link18
1 CacheCENICRouter_Link18 CacheCENICRouter_Link18
['ESNET_Link4', 'CacheCENICRouter_Link18']
True
0 ESNET_Link4 CacheCENICRouter_Link19
1 CacheCENICRouter_Link4 CacheCENICRouter_Link19
2 CacheCENICRouter_Link19 CacheCENICRouter_Link19
['ESNET_Link4', 'CacheCENICRouter_Link4', 'CacheCENICRouter_Link19']
True
0 CacheCENICRouter_Link4 CacheCENICRouter_Link4
['CacheCENICRouter_Link4']
True
0 ESNET_Link4 CacheCal
1 CacheCENICR

(52, 0.7878787878787878)